Testing Propotype with Selenium & google library
*** I have using FireFox for the selenium web driver. Be sure that Firefox has been installed on the local machine.

In [ ]:
pip install selenium pandas google


In [ ]:
#from googlesearch import search_news
from googlesearch import search
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from urllib.parse import urlparse
import pandas as pd
import time
import traceback


In [9]:
OUTPUT_FILE = 'output.csv'
AUTHOR_MAX_WORDS = 3
SLEEP_INTERVAL = 2

p_elementList= []

# Cuntion for Getting Date from Content
def extractDate(String):
    try:
        return False
    except:
        print("Error accessing a text:", e)
    return True

# Function for Getting Content from url
def getContent(driver,url):
    try:
        parsed_url = urlparse(url)

        authors = '' 
        source_domain = parsed_url.netloc
        copyright = ''
        paragraph = ''
        title = ''
        readMoreUrl = ''
        date = ''
        driver.get(url)
        H1_elements = driver.find_elements(By.TAG_NAME, "h1")
        p_elements = driver.find_elements(By.TAG_NAME, "p")
        #a_elements = driver.find_elements(By.TAG_NAME, "a")

        # process with Readmore link
        #for a in a_elements:
        #    try:
        #        value = a.text
        #        if value.casefold() == 'read more':
        #            readMoreUrl = a.get_attribute('href')
        #    except Exception as e:
        #        print("Error accessing a text:", e)
        #        continue
            
        # Getting Title
        for h1 in H1_elements:
            try:
                title = h1.text
                break  # Assuming you only need the first h1's text
            except Exception as e:
                print("Error accessing h1 text:", e)
                continue

        # Getting Content
        for p in p_elements:
            try:
                value = p.text
                # Filter out authors
                if value.startswith('By '):
                    authors = value.replace('By ', '') 
                    words = authors.split()
                    if len(words) <= AUTHOR_MAX_WORDS and not authors:
                        authors = value
                    else:
                        authors = ''
                    continue
                # Filter out Copyright
                if '©' in value or 'All Rights Reserved' in value:
                    copyright = value
                    continue

                # call date function to extract the date
                if (extractDate(value)):
                    date = value
                    continue

                paragraph += value + ' '
            except Exception as e:
                print(f"Error accessing paragraph text from {url}:\n", e)
                continue
    except TimeoutException:
        authors = ''
        print(f"Timeout waiting for elements on the page: {url}")
    except NoSuchElementException:
        authors = ''
        print(f"An element was not found on the page: {url}")
    except Exception as e:
        authors = ''
        print(f"An unexpected error occurred with URL => {url}: {e}")

    return {'authors': authors, 'date' : date, 'title' : title, 'content': paragraph, 'source_domain': source_domain, 'url' : url, 'copyright': copyright } #, 'readMoreUrl' : readMoreUrl }

def getPElementList(driver,url):
    try:
        parsed_url = urlparse(url)

        authors = '' 
        source_domain = parsed_url.netloc
        copyright = ''
        paragraph = ''
        title = ''
        readMoreUrl = ''
        date = ''
        driver.get(url)
        p_elements = driver.find_elements(By.TAG_NAME, "p")

        # Getting Content
        for p in p_elements:
            try:
                value = p.text
                p_elementList.append({'p_element': value,'title' : title,  'source_domain': source_domain, 'url' : url, })

            except Exception as e:
                print(f"Error accessing paragraph text from {url}:\n", e)
                continue
    except TimeoutException:
        authors = ''
        print(f"Timeout waiting for elements on the page: {url}")
    except NoSuchElementException:
        authors = ''
        print(f"An element was not found on the page: {url}")
    except Exception as e:
        authors = ''
        print(f"An unexpected error occurred with URL => {url}: {e}")

    return 

def save_to_csv_with_pandas(recordSet, filename):
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(recordSet)
    # Save the DataFrame to a CSV file
    df.to_csv(filename, index=False, encoding='utf-8')



In [12]:
# define query String
#queryString = 'Did Joe Biden tell voters to stay home and not vote?'
queryString = 'Is Michael Saylor saying he will give away free bitcoin?'

#Search in google url with google library
results = search(query=queryString,num = 10 )

In [10]:
driver = webdriver.Firefox()
i = 0
for url in results:
    print(f'Scripting from => {url}')
    try:
        getPElementList(driver,url)
    except Exception as e:
        print(f"An unexpected error occurred with URL => {url}: {e}")
    #time.sleep(SLEEP_INTERVAL)
driver.close()
df = pd.DataFrame(p_elementList)
df

Scripting from => https://www.reuters.com/world/us/fake-biden-robo-call-tells-new-hampshire-voters-stay-home-2024-01-22/
Scripting from => https://www.reuters.com/article/uk-fact-check-biden-voter-protection-not-idUSKBN27E2VH/
Scripting from => https://www.theguardian.com/us-news/video/2020/may/22/joe-biden-charlamagne-you-aint-black-trump-video
Scripting from => https://www.politico.com/news/2020/03/13/democrats-confront-a-never-biden-contingent-127438
Scripting from => https://www.nbcnews.com/politics/2024-election/fake-joe-biden-robocall-tells-new-hampshire-democrats-not-vote-tuesday-rcna134984
Scripting from => https://www.cnn.com/2022/01/20/politics/fact-check-biden-false-claims-first-year-2021/index.html
Scripting from => https://www.latimes.com/opinion/story/2020-05-22/biden-says-if-youre-black-and-dont-vote-for-him-youre-not-black-hes-right
Scripting from => https://www.whitehouse.gov/briefing-room/speeches-remarks/2022/01/11/remarks-by-president-biden-on-protecting-the-right-t

,p_element,title,source_domain,url
0,Reporting by Steve Holland; Additional reporti...,,www.reuters.com,https://www.reuters.com/world/us/fake-biden-ro...
1,Our Standards: The Thomson Reuters Trust Princ...,,www.reuters.com,https://www.reuters.com/world/us/fake-biden-ro...
2,Donald Trump's ex-lawyer Rudy Giuliani on Tues...,,www.reuters.com,https://www.reuters.com/world/us/fake-biden-ro...
3,New York State Attorney General Letitia James ...,,www.reuters.com,https://www.reuters.com/world/us/fake-biden-ro...
4,"The legal team for Hunter Biden, U.S. Presiden...",,www.reuters.com,https://www.reuters.com/world/us/fake-biden-ro...
...,...,...,...,...
5693,Sign up for free newsletters and get more CNBC...,,www.cnbc.com,https://www.cnbc.com/2022/12/09/majority-of-am...
5694,"Get this delivered to your inbox, and more inf...",,www.cnbc.com,https://www.cnbc.com/2022/12/09/majority-of-am...
5695,© 2024 CNBC LLC. All Rights Reserved. \nA Divi...,,www.cnbc.com,https://www.cnbc.com/2022/12/09/majority-of-am...
5696,Data is a real-time snapshot *Data is delayed ...,,www.cnbc.com,https://www.cnbc.com/2022/12/09/majority-of-am...


In [16]:
dataSet = []
driver = webdriver.Firefox()
i = 0
for url in results:
    print(f'Scripting from => {url}')
    try:
        dataSet.append(getContent(driver,url))
    except:
        print(f"An unexpected error occurred with URL => {url}: {e}")
    #time.sleep(SLEEP_INTERVAL)
driver.close()
df = pd.DataFrame(dataSet)


In [15]:
df.head(5)

,0
0,None
1,None
2,None
3,None
4,None


In [11]:
save_to_csv_with_pandas(df, queryString)